# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [38]:
# Google Chrome -> chromedriver

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys

# para rodar o chrome em 2° plano
#from selenium.webdriver.chrome.options import Options
#chrome_options = Options()
#chrome_options.headless = True
#nav = webdriver.Chrome(options=chrome_options)

# abrir um navegador
navegador = webdriver.Chrome()

navegador.get('https://www.google.com/')

# Passo 1: Pegar a cotação do Dólar
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Passo 2: Pegar a cotação do Euro
navegador.get('https://www.google.com/')

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Passo 3: Pegar a cotação do Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [39]:
# Passo 4: Importar a Lista de produtos
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [40]:
# Passo 5: Recalcular o preço de cada produto

# Atualizar a cotação
# nas linhas onde na coluna "Moeda" = dolar
tabela.loc[tabela['Moeda'] == 'Dólar', "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', "Cotação"] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', "Cotação"] = float(cotacao_ouro)

# Atualizar o preço base reais (preço original * cotação)
tabela['Preço Base Reais'] = tabela['Preço Base Original'] * tabela['Cotação']

# Atualizar o preço final (preço base reais * Margem)
tabela['Preço Final'] = tabela['Preço Base Reais'] * tabela['Margem']

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.934500,4934.450655,1.40,6908.230917
1,Carro Renault,4500.00,Euro,5.889869,26504.408453,2.00,53008.816905
2,Notebook Dell,899.99,Dólar,4.934500,4441.000655,1.70,7549.701113
3,IPhone,799.00,Dólar,4.934500,3942.665500,1.70,6702.531350
4,Carro Fiat,3000.00,Euro,5.889869,17669.605635,1.90,33572.250706
5,Celular Xiaomi,480.48,Dólar,4.934500,2370.928560,2.00,4741.857120
6,Joia 20g,20.00,Ouro,282.620000,5652.400000,1.15,6500.260000


### Agora vamos exportar a nova base de preços atualizada

In [42]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel('Produtos Novo.xlsx', index=False)